In [1]:
import pandas as pd
import numpy as np
import faiss
import time

In [2]:
#pd.set_option('display.max_rows', None)

In [3]:
#pd.reset_option('display.max_rows')

In [4]:
#d = pd.read_csv('data/tokens.txt', header=None, delimiter=' ', quoting=3)

In [5]:
#d.to_pickle('data/tokens.pkl')

In [6]:
d = pd.read_pickle('data/tokens.pkl')

In [7]:
#used to test after reading tokens.txt to see if I had done it right
#then used to see what the first ten words were
print(d.iloc[:21, :3])

       0         1         2
0    the  0.306717 -0.320530
1      ,  0.022532  0.320788
2      . -0.075626  0.369955
3     of  0.487177 -0.193218
4     to  0.173940 -0.104660
5    and  0.222883 -0.001161
6     in  0.712315  0.451855
7      a  0.048874  0.379618
8      "  0.019833  0.136752
9      - -0.308798 -0.711716
10    's  0.777245 -0.275744
11   for  0.312680  0.062339
12  that  0.465040 -0.383865
13    on  0.285893 -0.133414
14   was  0.444401  0.233967
15    is  0.826911 -0.049384
16  said -0.434792  0.498616
17  with  0.213500  0.433140
18    he  0.753313  0.336813
19    as  0.414907 -0.157948
20    it  0.401223  0.298163


In [8]:
first_ten_words = d.iloc[[0,3,4,5,6,7,11,12,13,14]]

In [9]:
#cosine similarity = v1 dot v2 / (norm(v1) * norm(v2))
#cosine distance = 1 - cosine similarity

cos_similarities = np.empty((10,10))
for i in range(10):
    for j in range (10):
        cos_similarities[i, j] = np.dot(first_ten_words.iloc[i, 1:], first_ten_words.iloc[j, 1:]) / (np.linalg.norm(first_ten_words.iloc[i, 1:]) * np.linalg.norm(first_ten_words.iloc[j, 1:]))
cos_similarities = np.clip(cos_similarities, -1, 1) #clipped to fix a rounding error
cos_distances = 1 - cos_similarities

print(np.round(cos_distances, decimals=3))

[[0.    0.145 0.199 0.205 0.153 0.18  0.182 0.138 0.155 0.202]
 [0.145 0.    0.271 0.191 0.173 0.209 0.187 0.217 0.231 0.288]
 [0.199 0.271 0.    0.194 0.218 0.25  0.163 0.164 0.183 0.322]
 [0.205 0.191 0.194 0.    0.172 0.262 0.144 0.172 0.176 0.273]
 [0.153 0.173 0.218 0.172 0.    0.189 0.179 0.203 0.17  0.2  ]
 [0.18  0.209 0.25  0.262 0.189 0.    0.17  0.189 0.21  0.216]
 [0.182 0.187 0.163 0.144 0.179 0.17  0.    0.164 0.138 0.246]
 [0.138 0.217 0.164 0.172 0.203 0.189 0.164 0.    0.189 0.235]
 [0.155 0.231 0.183 0.176 0.17  0.21  0.138 0.189 0.    0.236]
 [0.202 0.288 0.322 0.273 0.2   0.216 0.246 0.235 0.236 0.   ]]


In [10]:
#angular distance = arccos(v1 dot v2 / (norm(v1) * norm(v2)))

angular_distances = np.arccos(cos_similarities)

print(np.round(angular_distances, decimals=3))

[[0.    0.545 0.642 0.652 0.56  0.61  0.613 0.532 0.564 0.647]
 [0.545 0.    0.753 0.628 0.598 0.658 0.622 0.671 0.693 0.779]
 [0.642 0.753 0.    0.633 0.673 0.723 0.579 0.581 0.614 0.826]
 [0.652 0.628 0.633 0.    0.595 0.741 0.544 0.596 0.602 0.757]
 [0.56  0.598 0.673 0.595 0.    0.626 0.607 0.649 0.592 0.644]
 [0.61  0.658 0.723 0.741 0.626 0.    0.591 0.625 0.661 0.669]
 [0.613 0.622 0.579 0.544 0.607 0.591 0.    0.581 0.531 0.717]
 [0.532 0.671 0.581 0.596 0.649 0.625 0.581 0.    0.624 0.7  ]
 [0.564 0.693 0.614 0.602 0.592 0.661 0.531 0.624 0.    0.701]
 [0.647 0.779 0.826 0.757 0.644 0.669 0.717 0.7   0.701 0.   ]]


In [11]:
def angular_distance(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    if norm1 == 0 or norm2 == 0:
        return np.pi  # max angle if either vector is zero

    cosine_sim = dot_product / (norm1 * norm2)
    cosine_sim = np.clip(cosine_sim, -1.0, 1.0)

    return np.arccos(cosine_sim)  # raw angle in radians

In [12]:
ang_dis_arr_rounded_no_repeats = np.tril(np.round(angular_distances, decimals=4))
ang_dis_df = pd.DataFrame(columns=['word1', 'word2', 'angular distance'])
for i in range(10):
    for j in range(10):
        if ang_dis_arr_rounded_no_repeats[i,j] != 0:
            ang_dis_df.loc[len(ang_dis_df)] = {'word1': first_ten_words.iloc[i,0], 'word2': first_ten_words.iloc[j,0], 'angular distance': ang_dis_arr_rounded_no_repeats[i,j]}

ang_dis_dif = ang_dis_df.sort_values('angular distance')
print(ang_dis_dif)

   word1 word2  angular distance
34    on   for            0.5311
21  that   the            0.5319
18   for   and            0.5438
0     of   the            0.5455
6     in   the            0.5601
28    on   the            0.5639
17   for    to            0.5790
27  that   for            0.5813
23  that    to            0.5814
20   for     a            0.5908
32    on    in            0.5924
9     in   and            0.5945
24  that   and            0.5961
7     in    of            0.5979
31    on   and            0.6021
19   for    in            0.6070
10     a   the            0.6097
15   for   the            0.6126
30    on    to            0.6145
16   for    of            0.6223
35    on  that            0.6245
26  that     a            0.6250
14     a    in            0.6257
4    and    of            0.6276
5    and    to            0.6329
1     to   the            0.6416
40   was    in            0.6437
36   was   the            0.6470
25  that    in            0.6493
3    and  

In [13]:
#for comparison (make sure the list is the same with both distances measures)

cos_dis_arr_rounded_no_repeats = np.tril(np.round(cos_distances, decimals=4))
cos_dis_df = pd.DataFrame(columns=['word1', 'word2', 'cosine distance'])
for i in range(10):
    for j in range(10):
        if cos_dis_arr_rounded_no_repeats[i,j] != 0:
            cos_dis_df.loc[len(cos_dis_df)] = {'word1': first_ten_words.iloc[i,0], 'word2': first_ten_words.iloc[j,0], 'cosine distance': cos_dis_arr_rounded_no_repeats[i,j]}

cos_dis_dif = cos_dis_df.sort_values('cosine distance')
print(cos_dis_dif)

   word1 word2  cosine distance
34    on   for           0.1378
21  that   the           0.1382
18   for   and           0.1442
0     of   the           0.1451
6     in   the           0.1528
28    on   the           0.1548
17   for    to           0.1630
27  that   for           0.1642
23  that    to           0.1643
20   for     a           0.1695
32    on    in           0.1704
9     in   and           0.1716
24  that   and           0.1725
7     in    of           0.1735
31    on   and           0.1759
19   for    in           0.1787
10     a   the           0.1802
15   for   the           0.1818
30    on    to           0.1829
16   for    of           0.1875
35    on  that           0.1887
26  that     a           0.1891
14     a    in           0.1894
4    and    of           0.1905
5    and    to           0.1937
1     to   the           0.1989
40   was    in           0.2001
36   was   the           0.2021
25  that    in           0.2035
3    and   the           0.2050
11     a

In [14]:
# QUESTION 2

In [15]:
#2A
start_time = time.time()

embeddings = d.iloc[:100000, 1:].to_numpy().astype('float32')
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

end_time = time.time()
print('Time for operation: ', end_time - start_time)

Time for operation:  0.7488148212432861


In [16]:
print('Number of vectors in index:', index.ntotal)

Number of vectors in index: 100000


In [17]:
print('Embedding array size (MB):', embeddings.nbytes / (1024*1024))

Embedding array size (MB): 38.14697265625


In [18]:
#2B and C (just change n_neighbors_to_fetch)
#refers to the row index from the previous cell as the query for index.search()
#the is the number of neighbors to return
query_word = 'data'
n_neighbors_to_fetch = 31

start_time = time.time()

query_index = d[d[0]==query_word].index[0]
query_word_token = embeddings[query_index : query_index+1]
distances_to_knn, indices_of_knn = index.search(query_word_token, n_neighbors_to_fetch)

end_time = time.time()
print('Time for operation: ', end_time - start_time)

Time for operation:  0.0861518383026123


In [19]:
distances_to_knn[0]

array([ 0.      , 18.257359, 19.054846, 21.089666, 21.189434, 23.210901,
       23.637829, 24.611519, 24.932158, 25.014446, 25.301332, 25.402464,
       26.047045, 26.098978, 26.23256 , 26.469494, 27.03086 , 27.048824,
       27.206549, 27.261578, 27.412685, 27.416449, 27.439491, 27.583212,
       27.586948, 27.662615, 27.700062, 27.753094, 27.951813, 28.021767,
       28.038372], dtype=float32)

In [20]:
closest = pd.DataFrame(columns=['word','Euclidean distance'])
closest['word'] = d.iloc[indices_of_knn[0],0]
closest['Euclidean distance'] = distances_to_knn[0]
print(closest)

                 word  Euclidean distance
943              data            0.000000
447       information           18.257359
2535         analysis           19.054846
6716         tracking           21.089666
6782         database           21.189434
3605     applications           23.210901
1361          numbers           23.637829
4490         indicate           24.611519
1530          account           24.932158
1181         provided           25.014446
5950       indicating           25.301332
836         available           25.402464
15598       databases           26.047045
990          computer           26.098978
5434       comparison           26.232559
2164         accounts           26.469494
1223          systems           27.030861
11444        analyzed           27.048824
2307       statistics           27.206549
5635        indicates           27.261578
14160        analyses           27.412685
2558       electronic           27.416449
6349   identification           27

In [21]:
#first attempt at 2D, but it turns out I misunderstood the assignment and didn't need to do this
#exact duplicate of code from a few cells above, I just wanted to make it a new cell because everything above here works perfectly

#query_word = 'data'
#n_neighbors_to_fetch = 100000

#start_time = time.time()

#query_index = d[d[0]==query_word].index[0]
#query_word_token = embeddings[query_index : query_index+1]
#distances_to_knn, indices_of_knn = index.search(query_word_token, n_neighbors_to_fetch)

#end_time = time.time()
#print('Time for operation: ', end_time - start_time)

#closest = pd.DataFrame(columns=['word','Euclidean distance'])
#closest['word'] = d.iloc[indices_of_knn[0][1:11]][0]
#closest['Euclidean distance'] = distances_to_knn[0][1:11]
#print(closest)

In [25]:
#2D

query_word = 'data'
query_index = d[d[0]==query_word].index[0]
words = d.iloc[:100000,0].to_numpy()
tokens = d.iloc[:100000, 1:].to_numpy()
euc_dist = np.empty(100000)
query_token = tokens[query_index]

starttime = time.time()
for i in range(len(words)):
    euc_dist[i] = np.linalg.norm(tokens[i] - query_token)
endtime = time.time()

distances_df = pd.DataFrame({'word':words, 'Euclidean distance from "data"':euc_dist})
distances_df = distances_df.sort_values('Euclidean distance from "data"')
print(distances_df.head(15))
print(endtime-starttime)

               word  Euclidean distance from "data"
943            data                        0.000000
447     information                        4.272863
2535       analysis                        4.365185
6716       tracking                        4.592349
6782       database                        4.603198
3605   applications                        4.817769
1361        numbers                        4.861875
4490       indicate                        4.961000
1530        account                        4.993211
1181       provided                        5.001444
5950     indicating                        5.030043
836       available                        5.040086
15598     databases                        5.103630
990        computer                        5.108716
5434     comparison                        5.121773
0.3712325096130371


In [41]:
#QUESTION 3

#3A

def angular_similarity(v1, v2):
    return 1 - 1/np.pi*np.arccos(np.dot(v1/np.linalg.norm(v1), v2/np.linalg.norm(v2)))

In [46]:
#3A

query_word1 = 'data'
query_word2 = 'mining'
query1_index = d[d[0]==query_word1].index[0]
query2_index = d[d[0]==query_word2].index[0]
query1_token = d.iloc[query1_index,1:]
query2_token = d.iloc[query2_index,1:]

print(angular_similarity(query1_token, query2_token))

0.6155021824750988


In [48]:
#3B

import numpy as np
random_vector = np.random.randn(100)
unit_vector = random_vector / np.linalg.norm(random_vector)
print(angular_similarity(query1_token, unit_vector))
print(angular_similarity(query2_token, unit_vector))

0.46301423043225065
0.5315541098203352


In [52]:
#3C
number_of_random_vectors = 10

dprods_q1 = np.empty(number_of_random_vectors)
dprods_q2 = np.empty(number_of_random_vectors)
rvecs = np.empty((number_of_random_vectors,100))

for i in range(number_of_random_vectors):
    random_vector = np.random.randn(100)
    unit_vector = random_vector / np.linalg.norm(random_vector)
    rvecs[i] = unit_vector
    dprods_q1[i] = np.dot(unit_vector, query1_token)
    dprods_q2[i] = np.dot(unit_vector, query2_token)

print(dprods_q1)
print(dprods_q2)